In [ ]:
import http.client
import json
import unicodedata

access_token = """OwNpo00TnGOaT4VceqO4DRHynPGnXypQWLbSjtwI8E8aC.99qUzcjMum\
iOTiZUYCWIms8msjfJVvN5uNcfAzLlxkAe25emA4GUcWROXgCqJf8o1VNEm6hxkX3hJ260qR"""
conn = http.client.HTTPSConnection("api.surveymonkey.com")
headers = {
    'Accept': "application/json",
    'Authorization': "Bearer %s" % access_token
}

old_surveys0 = []

In [ ]:
open_ended_response0 = []
single_choice_response0 = []
multiple_choice_response0 = []
matrix_response0 = []
demographic_response0 = []
datetime_response0 = []

In [ ]:
# Retrieve all survey ids and titles
def surveys():

    conn.request("GET", "/v3/surveys", headers=headers)
    res = conn.getresponse()
    data = json.load(res)['data']
    
    svy_lst = []

    for survey in data:
        surv = {"sid" : int(survey["id"]),
                "svy_ttl" : unicodedata.normalize("NFKD", survey["title"]),
                "svy_link" : survey["href"]
               }
        svy_lst.append(surv)

    return svy_lst

In [ ]:
# Save all new survey ids to new_surveys0
surveys_backkup0 = old_surveys0.copy()
new_surveys0 = [survey for survey in surveys() if survey not in old_surveys0]
old_surveys0 = old_surveys0 + new_surveys0

In [ ]:
# Retrieve survey detail in a specified survey
def survey_data(survey_id):
    
    conn.request("GET", "/v3/surveys/%s/details" % survey_id, headers=headers)
    res = conn.getresponse()
    data = json.load(res)
    
    return data

In [ ]:
# Retrieve all survey detail from all surveys
def all_survey_data():
    
    survey_data_list = []
    
    for svy in new_surveys0:
        survey_id = svy["sid"]
        d = survey_data(survey_id)
        survey_data_list.append(survey_data(survey_id))
        
    return survey_data_list

In [ ]:
# Save all question data in all_survey_detail0
all_survey_data0 = all_survey_data()

In [ ]:
all_survey_data0_backup0 = all_survey_data0.copy()

In [ ]:
# Retrieve the response data in a specified survey
def response_data(survey_id):
    conn.request("GET", "/v3/surveys/%s/responses/bulk" % survey_id, headers=headers)
    res = conn.getresponse()
    data = json.load(res)
    
    return data

In [ ]:
# Retrieve all response data from all surveys
def all_response_data():
    
    response_data_list = []
    
    for svy in new_surveys0:
        survey_id = svy["sid"]
        response_data_list.append(response_data(survey_id))
        
    return response_data_list

In [ ]:
# Save all response data in all_response_data0
all_response_data0 = all_response_data()

In [ ]:
all_response_data0_backup0 = all_response_data0.copy()

In [ ]:
# Retrieve all required question info in a specificed survey
def question(survey_id: int) -> dict:
    
    data = ""
    question_list = []
    family_table = {"single_choice" : 1,
                    "matrix" : 2,
                    "open_ended" : 3,
                    "demographic" : 4,
                    "datetime" : 5,
                    "multiple_choice" : 6,
                    "presentation" : 7
                   }
    # First digit in the subtype_id in the subtype_table 
    # refers to the family_id in the family_table
    single_choice_subtype_table = {"vertical" : 11,
                     "horiz" : 12,
                     "menu" : 13,
                     "vertical_two_col" : 14}
    matrix_subtype_table = {"single" : 21,
                            "rating" : 22,
                            "ranking" : 23,
                            "menu" : 24,
                            "multi": 25}
    open_ended_subtype_table = {"single" : 31,
                                "multi" : 32,
                                "numerical" : 33,
                                "essay" : 34}
    demographic_subtype_table = {"international" : 41,
                                 "us" : 42}
    datetime_subtype_table = {"both" : 51,
                              "date_only" : 52,
                              "time_only" : 53}
    multiple_choice_subtype_table = {"vertical" : 61,
                                     "vertical_two_col" : 62}
    presentation_subtype_table = {"descriptive_text" :71,
                                  "image" : 72}
    
    qtn_info = {"sid" : survey_id,
              "pos" : "",
              "qid" : "",
              "question" : "",
              "fid" : "",
              "stid" : "",
             }
    
    for survey_data in all_survey_data0:
        if int(survey_data["id"]) == survey_id:
            data = survey_data
            break
                
    pages = data["pages"]
    question_count = 0
    
    for page in pages:
        qtns = page["questions"]
        if not qtns: # qtns is empty
            question_list.append(qtn_info)
            return question_list
        for qtn in qtns:
            qtn_info["pos"] = qtn["position"] + question_count
            qtn_info["qid"] = int(qtn["id"])
            qtn_info["question"] = unicodedata.normalize("NFKD", qtn["headings"][0]["heading"])
            qtn_info["fid"] = family_table[qtn["family"]]
            if qtn_info["fid"] == 1:
                qtn_info["stid"] = single_choice_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 2:
                qtn_info["stid"] = matrix_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 3:
                qtn_info["stid"] = open_ended_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 4:
                qtn_info["stid"] = demographic_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 5:
                qtn_info["stid"] = datetime_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 6:
                qtn_info["stid"] = multiple_choice_subtype_table[qtn["subtype"]]
            elif qtn_info["fid"] == 7:
                qtn_info["stid"] = presentation_subtype_table[qtn["subtype"]]
            temp_qtn_info = qtn_info.copy()
            question_list.append(temp_qtn_info)
        question_count += page["question_count"]
    
    return question_list

In [ ]:
# Retrieve all required question info from all surveys
def questions():
    # Create an empty questions list
    qtns_lst = []
    
    for survey in new_surveys0:
        # Retrieve survey_id from survey
        survey_id = survey["sid"]
        # Add question_info to the questions list
        qtns_lst.append(question(survey_id))
        
    return qtns_lst

In [ ]:
questions0 = questions()

In [ ]:
questions0_backup_0 = questions0.copy()

In [ ]:
# Check question family with given survey_id and question_id
def question_family(survey_id: int, question_id: int) -> int:
    
    for q in question(survey_id):
        if q["qid"] == question_id:
            return q["fid"]

In [ ]:
def response(survey_id: int) -> dict:
    
    pages = ""
    q_family = ""
    response_list = []
    
    for response_data in all_response_data0:
        data = response_data["data"]
        for resp in data:
            if int(resp["survey_id"]) == survey_id:                
                pages = resp["pages"]
                response_id = int(resp["id"])
            else:
                continue
    
            for page in pages:
                r_questions = page["questions"]
                for r_question in r_questions:
                    question_id = int(r_question["id"])
                    answers = r_question["answers"]
                    q_family = question_family(survey_id, question_id)
                                        
                    # open_ended
                    if q_family == 3:
                        text = unicodedata.normalize("NFKD", answers[0]["text"])
                        resp_info = {
                            "rid" : response_id,
                            "qid" : question_id,
                            "text" : text
                        }
                        open_ended_response0.append(resp_info)
                    
                    # single_choice
                    if q_family == 1:                                                
                        for answer in answers:
                            cid = ""
                            oid = ""
                            otext = ""
                            if "other_id" in answer.keys():
                                oid = int(answer["other_id"])
                                otext = unicodedata.normalize("NFKD", answer["text"])
                            else:
                                cid = int(answers[0]["choice_id"])
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id,
                                        "cid" : cid,
                                        "oid" : oid,
                                        "otext" : otext
                                        }
                            single_choice_response0.append(resp_info)
                    
                    # multiple_choice
                    if q_family == 6:                        
                        for answer in answers:
                            cid = ""
                            oid = ""
                            otext = ""
                            if "other_id" in answer.keys():
                                oid = int(answer["other_id"])
                                otext = unicodedata.normalize("NFKD", answer["text"])
                            else:
                                cid = int(answer["choice_id"])
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id,
                                        "cid" : cid,
                                        "oid" : oid,
                                        "otext" : otext
                                        }    
                            multiple_choice_response0.append(resp_info)
                    
                    # matrix
                    if q_family == 2:                        
                        for answer in answers:
                            cid = ""
                            roid = ""
                            oid = ""
                            otext = ""
                            weight = ""
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id                                        
                                    }
                            if "choice_id" in answer.keys():
                                cid = int(answer["choice_id"])
                                roid = int(answer["row_id"])
                            if "other_id" in answer.keys():
                                oid = int(answer["other_id"])
                                otext = answer["text"]
                            if "choice_metadata" in answer.keys():
                                weight = int(answer["choice_metadata"]["weight"])
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id,
                                        "cid" : cid,
                                        "roid" : roid,
                                        "oid" : oid,
                                        "otext" : otext,
                                        "weight" : weight
                                        }    
                            matrix_response0.append(resp_info)
                    
                    # demographic
                    if q_family == 4:                        
                        for answer in answers:
                            roid = ""
                            rotext = ""
                            roid = int(answer["row_id"])
                            rotext = unicodedata.normalize("NFKD", answer["text"])
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id,
                                        "roid" : roid,
                                        "rotext" : rotext
                                    }
                            demographic_response0.append(resp_info)
                    
                    # datetime
                    if q_family == 5:                        
                        for answer in answers:
                            roid = ""
                            rotext = ""
                            roid = int(answer["row_id"])
                            rotext = answer["text"]
                            resp_info = {
                                        "rid" : response_id,
                                        "qid" : question_id,
                                        "roid" : roid,
                                        "rotext" : rotext
                                    }
                            datetime_response0.append(resp_info)
        
    return

In [ ]:
# Retrieve all required responses info in all surveys
def responses():
    
    resp_lst = []
    
    for survey in new_surveys0:
        survey_id = survey["sid"]
        print(survey_id)
        resp_lst.append(response(survey_id))
        
    return resp_lst

In [ ]:
responses0 = responses()

In [ ]:
surveys_backkup0 = old_surveys0.copy()